# 🧠 Cours complet : Les Processus Gaussiens (GP)

---

## 📘 1. Introduction

Un **processus gaussien (GP)** est une généralisation du concept de **loi normale** aux **fonctions continues**.

Alors qu’une loi normale décrit la distribution d’un **scalaire aléatoire** ou d’un **vecteur aléatoire** fini,  
un processus gaussien décrit une **distribution sur des fonctions**.

Autrement dit :

> Un processus gaussien est une distribution de probabilité sur l’ensemble des fonctions continues.  
> Il permet de dire « quelle forme une fonction peut prendre », tout en mesurant l’incertitude associée.





Les processus gaussiens (GP) sont une méthode d'apprentissage supervisé non paramétrique utilisée pour résoudre les problèmes de régression et de classification probabiliste.

Les avantages des processus gaussiens sont :

- La prédiction interpole les observations (au moins pour les noyaux réguliers).
- La prédiction est probabiliste (gaussienne) de sorte que l'on peut calculer des intervalles de confiance empiriques et décider en fonction de ceux-ci si l'on doit réajuster (ajustement en ligne, ajustement adaptatif) la prédiction dans une région d'intérêt.
- Polyvalent : différents kernels peuvent être spécifiés. Des noyaux courants sont fournis, mais il est également possible de spécifier des noyaux personnalisés.

Les inconvénients des processus gaussiens incluent :

- Notre implémentation n'est pas éparse, c'est-à-dire qu'elle utilise l'ensemble des informations sur les échantillons/caractéristiques pour effectuer la prédiction.
- Ils perdent en efficacité dans les espaces de grande dimension, notamment lorsque le nombre de caractéristiques dépasse quelques dizaines.

---

## 🧩 2. Définition formelle

On note un processus gaussien comme :

$$
f(x) \sim \mathcal{GP}(m(x), k(x, x'))
$$

où :

- $m(x) = \mathbb{E}[f(x)]$ est la **fonction moyenne**
- $k(x, x') = \mathrm{Cov}(f(x), f(x'))$ est la **fonction de covariance** (ou **kernel**)

Pour tout ensemble fini de points $X = [x_1, \dots, x_n]$,  
le vecteur aléatoire $f = [f(x_1), \dots, f(x_n)]^T$ suit une **loi normale multivariée** :

$$
f \sim \mathcal{N}(m(X), K(X, X))
$$

avec :

$$
m(X) = [m(x_1), \dots, m(x_n)]^T, \quad K(X, X)_{ij} = k(x_i, x_j)
$$

---

## 🎯 3. Cas unidimensionnel (1D)

Dans le cas 1D, on suppose que la fonction $f : \mathbb{R} \to \mathbb{R}$ est un processus gaussien.

Chaque paire de points $(x_i, x_j)$ est corrélée selon la covariance $k(x_i, x_j)$.

La distribution jointe sur $n$ points est alors :

$$
p(f(X)) = \frac{1}{(2\pi)^{n/2} |K|^{1/2}}
\exp\left(
-\frac{1}{2}(f(X) - m(X))^T K^{-1} (f(X) - m(X))
\right)
$$

où $K = K(X, X)$.

---

## 🔊 4. Cas avec bruit d’observation

Dans un problème réel, les observations $y_i$ sont bruitées :

$$
y_i = f(x_i) + \varepsilon_i, \quad \varepsilon_i \sim \mathcal{N}(0, \sigma_n^2)
$$

Cela signifie que la variance observée contient le **bruit de mesure** :

$$
y \sim \mathcal{N}\big(m(X), K(X, X) + \sigma_n^2 I\big)
$$

---

## 📈 5. Prédiction d’un nouveau point $x_*$

On veut connaître la distribution de $f_* = f(x_*)$ conditionnellement aux observations $(X, y)$.

On définit :

$$
k_* = k(X, x_*) \in \mathbb{R}^n, \quad
k_{**} = k(x_*, x_*), \quad
K_y = K(X, X) + \sigma_n^2 I
$$

La loi jointe entre $y$ et $f_*$ est :

$$
\begin{bmatrix}
y \\
f_*
\end{bmatrix}
\sim
\mathcal{N}\left(
\begin{bmatrix}
m(X) \\
m(x_*)
\end{bmatrix},
\begin{bmatrix}
K_y & k_* \\
k_*^T & k_{**}
\end{bmatrix}
\right)
$$

En conditionnant sur $y$, on obtient la loi a posteriori :

$$
f_* \,|\, X, y, x_* \sim \mathcal{N}(\mu_*, \sigma_*^2)
$$

avec :

$$
\mu_* = m(x_*) + k_*^T K_y^{-1} (y - m(X))
$$

et

$$
\sigma_*^2 = k_{**} - k_*^T K_y^{-1} k_*
$$

---

## 🧮 6. Fonction de vraisemblance marginale

Le modèle complet induit une **distribution sur les observations** :

$$
p(y | X, \theta) = \mathcal{N}\big(0, K_\theta + \sigma_n^2 I\big)
$$

La **log-vraisemblance marginale** s’écrit :

$$
\log p(y|X,\theta) =
-\frac{1}{2} y^T (K_\theta + \sigma_n^2 I)^{-1} y
-\frac{1}{2} \log |K_\theta + \sigma_n^2 I|
-\frac{n}{2} \log(2\pi)
$$

En pratique, on maximise cette fonction pour trouver les **hyperparamètres optimaux** :

$$
\theta = \{ \ell, \sigma_f, \sigma_n \}
$$

où :

- $\ell$ = longueur caractéristique (lisse ou non)
- $\sigma_f$ = amplitude du signal
- $\sigma_n$ = niveau de bruit

---

## Fonctions de covariance (Kernels) dans les Processus Gaussiens 

Les **noyaux (ou fonctions de covariance)** définissent la structure de dépendance et les propriétés de lissage d’un **processus gaussien (GP)**.  
Le choix du noyau est **crucial** : il détermine la manière dont le modèle généralise et la forme des fonctions qu’il peut apprendre.

---

### ⚙️ Rôle du noyau
Un **noyau** $k(x, x')$ mesure la **similarité** entre deux points d’entrée $x$ et $x'$.  
C’est lui qui encode nos **hypothèses a priori** sur la fonction $f(x)$ :
- sa **lissité**,
- sa **périodicité**,
- sa **linéarité**,
- ou encore la manière dont elle **varie localement**.

Un mauvais choix de noyau conduit souvent à un modèle mal calibré, qui surapprend ou sous-apprend.

---

### 🧩 Noyaux standards

#### 1. Noyau exponentiel quadratique (Squared Exponential ou RBF)
$$
k_{\text{SE}}(x, x') = \sigma^2 \exp\!\left( -\frac{(x - x')^2}{2\ell^2} \right)
$$
- Très utilisé car **universel** et **infinitement dérivable**.  
- Paramètres :
  - $\ell$ : *longueur de corrélation*, contrôle la largeur des "oscillations" (plus $\ell$ est petit, plus la fonction varie vite)  
  - $\sigma^2$ : *variance de sortie*, amplitude moyenne des fluctuations.  
- Idéal pour des fonctions **douces et continues**.  
- ⚠️ Moins adapté pour des fonctions avec discontinuités ou changements brusques.

---

#### 2. Noyau quadratique rationnel (Rational Quadratic)
$$
k_{\text{RQ}}(x, x') = \sigma^2 \left( 1 + \frac{(x - x')^2}{2 \alpha \ell^2} \right)^{-\alpha}
$$
- Modélise une **superposition** de plusieurs RBF à différentes échelles.  
- $\alpha$ contrôle le poids relatif entre les grandes et petites échelles :
  - $\alpha \to \infty$ : équivalent au SE kernel.  
- Bon pour des phénomènes à **plusieurs niveaux de variabilité**.

---

#### 3. Noyau périodique
$$
k_{\text{Per}}(x, x') = \sigma^2 \exp\!\left( -\frac{2\sin^2(\pi |x - x'| / p)}{\ell^2} \right)
$$
- Représente des fonctions **strictement périodiques**.  
- Paramètres :
  - $p$ : période du motif répété,  
  - $\ell$ : lissage local.  
- Exemple : variation saisonnière, signaux cycliques, etc.

---

#### 4. Noyau périodique local (Local Periodic)
$$
k_{\text{LocalPer}}(x, x') = k_{\text{Per}}(x, x') \cdot k_{\text{SE}}(x, x')
$$
- Combine périodicité + évolution lente du motif.  
- Représente des fonctions **périodiques mais évolutives** (ex. climat changeant d’une année à l’autre).

---

#### 5. Noyau linéaire
$$
k_{\text{Lin}}(x, x') = \sigma_b^2 + \sigma_v^2 (x - c)(x' - c)
$$
- Correspond à une **régression linéaire bayésienne**.  
- Non stationnaire (dépend de la position absolue).  
- Paramètres :
  - $c$ : centre de la régression (origine),
  - $\sigma_b^2$, $\sigma_v^2$ : contrôlent le biais et la variance des pentes.  
- Idéal pour des **tendances globales**.

---

### 🔗 Combinaison de noyaux

On peut **composer** des noyaux pour capturer plusieurs comportements à la fois.

#### ➕ Somme de noyaux
$$
k(x,x') = k_1(x,x') + k_2(x,x')
$$
- Interprétation : **OR logique** → le GP combine des comportements additifs.  
- Exemple : $k_{\text{SE}} + k_{\text{Per}}$ → fonction lisse + composante périodique.

#### ✖️ Produit de noyaux
$$
k(x,x') = k_1(x,x') \times k_2(x,x')
$$
- Interprétation : **AND logique** → les similarités doivent exister dans les deux composantes.  
- Exemple : $k_{\text{Per}} \times k_{\text{SE}}$ → périodique mais avec motifs qui évoluent lentement.

---

### 🧮 Propriétés utiles
- **Stationnarité** : $k(x, x') = k(x - x')$ dépend seulement de la distance → invariant par translation.  
- **Isotropie** : dépend uniquement de $||x - x'||$.  
- **Non stationnaire** : dépend de $x$ et $x'$ séparément (ex : linéaire).

---

### 🧠 Interprétation pratique

| Type de phénomène | Noyau recommandé | Exemple concret |
|--------------------|------------------|-----------------|
| Lisse et continu | SE | Température quotidienne |
| Multi-échelle | RQ | Marchés financiers |
| Périodique | Periodic | Données saisonnières |
| Périodique + dérive lente | Local Periodic | Cycle climatique changeant |
| Linéaire | Linear | Croissance d’une plante |
| Mixte | Somme/Produit | Signal composé (tendance + bruit + saisonnalité) |

---

### 🌍 Autres extensions

- **Variables catégorielles** : utiliser un *one-hot encoding* et un RBF sur chaque dimension.  
- **Symétrie connue** : on peut forcer $f(x) = f(-x)$ en ajoutant un noyau symétrisé :  
  $k_{\text{sym}}(x,x') = k(x,x') + k(-x,x')$  
- **Structure faible dimensionnelle** :  
  $k(x,x') = k(Ax, A x')$, avec $A$ une matrice de projection bas-dimensionnelle.

---

### 🧭 En résumé
- Le **noyau est le cœur du GP** : il définit le comportement de la fonction a priori.  
- Il faut **choisir ou construire** un noyau selon la **structure** du phénomène étudié.  
- Les combinaisons de noyaux permettent de modéliser des fonctions **complexes, mixtes, ou hiérarchiques**.  
- L’optimisation des **hyperparamètres** du noyau (via la log-vraisemblance marginale) ajuste automatiquement la flexibilité du modèle.





## 🌍 8. Extension en dimension n

Pour un espace d’entrée $x \in \mathbb{R}^d$ :

$$
f(x) \sim \mathcal{GP}(m(x), k(x, x'))
$$

avec par exemple le **RBF multidimensionnel** :

$$
k(x, x') = \sigma_f^2 \exp\left(-\frac{1}{2}(x - x')^T \Lambda^{-1} (x - x')\right)
$$

où :

$$
\Lambda = \text{diag}(\ell_1^2, \dots, \ell_d^2)
$$

Chaque dimension a sa propre **longueur caractéristique**.

---

## 🔍 9. Interprétation géométrique

Un processus gaussien peut être vu comme un **ensemble infini de fonctions** tirées d’une distribution conjointe.

Chaque fonction $f$ tirée d’un GP :

- respecte la moyenne $m(x)$,
- présente une « forme » compatible avec le kernel $k$.

Les zones avec peu de données ont une **incertitude élevée**,  
et celles avec beaucoup de points proches ont une **incertitude faible**.

---

## 📊 10. Applications concrètes

Les processus gaussiens sont utilisés dans :

| Domaine | Utilisation typique |
|----------|--------------------|
| **Régression** | Modéliser des relations non linéaires avec incertitude |
| **Optimisation bayésienne** | Chercher un optimum coûteux à évaluer |
| **Géostatistique (krigeage)** | Interpolation spatiale en géosciences |
| **Surrogate modeling** | Approximation de modèles coûteux (physique, CFD, etc.) |
| **Climatologie / écologie** | Modéliser des phénomènes spatiaux comme la croissance d’espèces |
| **Machine Learning bayésien** | Apprentissage probabiliste interprétable |

---

## 🌳 11. Exemple conceptuel : croissance d’espèces d’arbres

On suppose que la **hauteur moyenne d’un arbre** dépend :

- de son **âge** $x_1$,
- de la **température moyenne annuelle** $x_2$.

Le GP permet de modéliser :

$$
h(x_1, x_2) \sim \mathcal{GP}(m(x_1, x_2), k((x_1, x_2), (x_1', x_2')))
$$

où :

- $m(x_1, x_2)$ peut être pris nul ou linéaire  
- $k$ décrit la similarité écologique entre deux environnements

Cela permet :

- d’estimer la croissance attendue sous différents climats,
- de quantifier l’incertitude dans les zones peu observées,
- d’interpoler entre régions (krigeage écologique).

---

## 🧠 12. Résumé final

| Élément | Rôle |
|----------|------|
| $m(x)$ | Moyenne du processus |
| $k(x, x')$ | Covariance (similarité) |
| $\sigma_n^2$ | Variance du bruit |
| $\mu_*(x_*)$ | Moyenne prédictive |
| $\sigma_*^2(x_*)$ | Incertitude prédictive |
| Log-vraisemblance | Ajustement des hyperparamètres |
| Kernel | Encode la structure spatiale ou temporelle des données |

---

## 🧭 13. À retenir

> 🔹 Un **processus gaussien** est une distribution sur les fonctions.  
> 🔹 Il permet de **modéliser des phénomènes continus incertains** de manière non paramétrique.  
> 🔹 Il combine **interprétabilité**, **rigueur bayésienne** et **incertitude explicite**.  
> 🔹 Son cœur est le **kernel**, qui traduit nos hypothèses sur la régularité du monde réel.  
> 🔹 En pratique, il est fondamental pour la **régression probabiliste**, l’**optimisation bayésienne** et la **modélisation spatiale**.

---


# Python

## Régression par processus gaussien (RPG)

La **régression par processus gaussien (Gaussian Process Regression, GPR)** est implémentée en Python via `GaussianProcessRegressor` de `scikit-learn`. Elle permet de réaliser des prédictions probabilistes en fournissant à la fois la **moyenne** et l'**écart-type** de la prédiction.

---

## Principe général

- Un **processus gaussien (PG)** est défini par :
  - une **fonction a priori** (mean function) : constante et nulle (`normalize_y=False`) ou égale à la moyenne des données (`normalize_y=True`),
  - une **fonction de covariance** spécifiée par un **noyau (`kernel`)**.

- Le modèle combine cette fonction a priori avec la **vraisemblance des données d'apprentissage**.

---

## Optimisation des hyperparamètres

- Les hyperparamètres du noyau sont ajustés en maximisant la **vraisemblance logarithmique marginale (LML)** via l'**optimiseur** transmis à `GaussianProcessRegressor`.
- Pour éviter les **optima locaux**, l'optimisation peut être relancée plusieurs fois (`n_restarts_optimizer`) à partir de valeurs aléatoires dans les bornes autorisées.
- Si on ne souhaite **pas optimiser** les hyperparamètres initiaux, on peut passer `optimizer=None`.

---

## Gestion du bruit

- Le niveau de bruit des cibles peut être :
  - un **scalaire global**,
  - un **vecteur spécifique à chaque point** via `alpha`.
- Le bruit agit comme une **régularisation de Tikhonov**, ajouté à la diagonale de la matrice du noyau pour stabiliser l'ajustement.
- Alternative : inclure un **WhiteKernel** dans le noyau pour estimer le bruit global automatiquement.

---

## Fonctionnalités supplémentaires

- Prédiction **sans ajustement préalable** (basée uniquement sur le GP a priori).
- Méthode `sample_y(X)` : échantillonne les valeurs du GP (a priori ou a posteriori).
- Méthode `log_marginal_likelihood(theta)` : permet de tester d'autres méthodes d'optimisation des hyperparamètres (ex. MCMC).

---


## 🌐 Processus Gaussien en 2D (et en dimensions supérieures)

---

### 🎯 Intuition générale

Un **Processus Gaussien (GP)** définit une **distribution sur les fonctions**.  
On écrit :
$$
f(\mathbf{x}) \sim \mathcal{GP}(m(\mathbf{x}), k(\mathbf{x}, \mathbf{x}'))
$$

où :
- $m(\mathbf{x}) = \mathbb{E}[f(\mathbf{x})]$ est la **fonction moyenne**,  
- $k(\mathbf{x}, \mathbf{x}') = \text{Cov}(f(\mathbf{x}), f(\mathbf{x}'))$ est la **fonction de covariance (kernel)**.

En dimension 1, $\mathbf{x}$ est un scalaire.  
En **dimension 2**, $\mathbf{x} = (x_1, x_2)$ représente un **point dans le plan**,  
et $f(x_1, x_2)$ représente une **surface aléatoire**.

---

### 🧭 Processus Gaussien en 2D : la vision géométrique

Un GP 2D suppose que pour **tout ensemble fini de points d’entrée**  
$\{\mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_n\}$,  
le vecteur aléatoire des valeurs :
$$
\mathbf{f} = [f(\mathbf{x}_1), f(\mathbf{x}_2), \dots, f(\mathbf{x}_n)]^\top
$$
suit une **distribution normale multivariée** :
$$
\mathbf{f} \sim \mathcal{N}(\mathbf{m}, \mathbf{K})
$$

avec :
- $\mathbf{m} = [m(\mathbf{x}_1), \dots, m(\mathbf{x}_n)]^\top$  
- $\mathbf{K}_{ij} = k(\mathbf{x}_i, \mathbf{x}_j)$  

Ainsi, dans le cas 2D :
- Chaque point $(x_1, x_2)$ a une **valeur aléatoire corrélée** à ses voisins,
- Le **kernel** contrôle la *forme*, *la régularité*, et *l’étendue spatiale* de cette corrélation.

---

### 📈 Exemple d’interprétation visuelle

- Si $k$ est un **RBF kernel**, les points proches dans le plan $(x_1, x_2)$ auront des valeurs $f(x_1, x_2)$ similaires → surface lisse.  
- Si $k$ est un **périodique**, on obtient une surface avec des motifs répétitifs.  
- Si $k$ est **linéaire**, la surface sera un plan (ou légèrement inclinée selon les paramètres).

En Python, on peut visualiser un GP 2D comme une **carte de chaleur** ou une **surface 3D** :
- l’axe horizontal → $x_1$  
- l’axe vertical → $x_2$  
- la hauteur (ou couleur) → $f(x_1, x_2)$

---

### ⚙️ Ce qu’il se passe en pratique (dans Python)

#### 🧮 Étapes internes (ex : `sklearn.gaussian_process` ou `GPy`)

1. **Création du kernel**  
   Exemple : `kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=0.1)`

2. **Construction de la matrice de covariance**
   $$
   K(X, X) = \begin{bmatrix}
   k(x_1, x_1) & \cdots & k(x_1, x_n) \\
   \vdots & \ddots & \vdots \\
   k(x_n, x_1) & \cdots & k(x_n, x_n)
   \end{bmatrix}
   $$

3. **Calcul du modèle a posteriori**  
   Pour une observation bruitée :
   $$
   y_i = f(\mathbf{x}_i) + \varepsilon_i, \quad \varepsilon_i \sim \mathcal{N}(0, \sigma_n^2)
   $$
   on obtient :
   $$
   p(f_* | X, y, X_*) = \mathcal{N}(\mu_*, \Sigma_*)
   $$
   avec :
   $$
   \begin{aligned}
   \mu_* &= K(X_*, X)[K(X, X) + \sigma_n^2 I]^{-1} y \\
   \Sigma_* &= K(X_*, X_*) - K(X_*, X)[K(X, X) + \sigma_n^2 I]^{-1} K(X, X_*)
   \end{aligned}
   $$

4. **Optimisation automatique des hyperparamètres**  
   Les paramètres du kernel ($\ell, \sigma^2, \sigma_n^2$, etc.) sont ajustés en **maximisant la log-vraisemblance marginale** :
   $$
   \log p(y | X) = -\frac{1}{2} y^\top [K(X,X) + \sigma_n^2 I]^{-1} y - \frac{1}{2}\log|K(X,X) + \sigma_n^2 I| - \frac{n}{2}\log(2\pi)
   $$

   👉 Cela permet d’adapter la **complexité du modèle** à la structure réelle des données.

---

### 📊 En 2D et au-delà (n dimensions)

Le principe reste le même :  
$\mathbf{x} \in \mathbb{R}^d$, le noyau $k(\mathbf{x}, \mathbf{x}')$ définit la corrélation entre les points dans cet espace $d$-dimensionnel.

- En 3D → on apprend un **volume de corrélations**  
- En nD → on apprend une **surface implicite** dans un espace de grande dimension.

---

### 🧠 Interprétation pratique en 2D

| Type de surface observée | Noyau typique | Interprétation |
|----------------------------|----------------|----------------|
| Surface lisse | RBF | terrain, température, pression atmosphérique |
| Motif périodique | Periodic | vagues, climat, cycles biologiques |
| Gradient global | Linear | plan incliné, tendance spatiale |
| Surface évoluant localement | Local Periodic ou Sum | paysage avec motifs + bruit local |

---

### 💡 En résumé

- Le GP 2D modélise une **surface aléatoire corrélée** dans le plan.  
- Le **kernel** définit la *texture* et la *régularité* de cette surface.  
- En Python :
  - On construit un kernel adapté (RBF, Matern, etc.),
  - On entraîne le modèle (fit),
  - On prédit sur une grille 2D (predict),
  - Et on visualise la **moyenne prédite** et **l’incertitude** sous forme de carte ou de surface.

---

📘 **À retenir :**
> Un GP ne prédit pas une valeur, il prédit **une distribution** sur les fonctions possibles.  
> En 2D ou nD, cela revient à prédire une **surface probabiliste**, où chaque point a une moyenne et une variance qui reflète notre **confiance** dans la prédiction.

